# Data Science - Laboratorio 9
## Visualizaciones interactivas y dashboards
### Procesamiento de Datos
---
**Integrantes**
- Diego Alberto Leiva
- José Pablo Orellana

## Librerias

In [1]:
# Manipulacion de Datos
import pandas as pd

## Carga de Conjunto de Datos

In [2]:
# Leer los archivos de una carpeta y almacenarlos como dataframes
consumos = pd.read_csv('raw_data/consumos_combustibles.csv', sep=';', encoding='utf-8')
importaciones = pd.read_csv('raw_data/importaciones_combustibles.csv', sep=';', encoding='utf-8')
precios = pd.read_csv('raw_data/precios_combustibles.csv', sep=';', encoding='utf-8')

data_frames = {
    'Consumos': consumos,
    'Importaciones': importaciones,
    'Precios': precios
}

# Mostrar los nombres de los archivos
print(f"Se cargaron un total de 3 archivos\n")
print('Archivos cargados:')
for key in data_frames.keys():
    print(f"\t- {key}")

Se cargaron un total de 3 archivos

Archivos cargados:
	- Consumos
	- Importaciones
	- Precios


## Preprocesamiento

### Seleccion de Variables

Solo se tomarán en cuenta los datos de importación, consumo y precios de los siguientes tipos de combustibles:
- `Gasolina Súper`
- `Gasolina Regular`
- `Diesel` -> combinacion de Diesel alto azufre y bajo azufre

In [3]:
# Seleccionar las columnas de interés para consumos e importaciones
cols = ['Fecha','Gasolina regular', 'Gasolina superior', 'Diesel alto azufre', 'Diesel bajo azufre']

# Filtrar columnas en 'consumo_combustibles'
consumos = consumos[cols].copy()

# Filtrar columnas en 'importacion_combustibles'
importaciones = importaciones[cols].copy()

# Filtrar columnas en 'precios_combustibles'
precios_cols = ['FECHA','Superior GTQ/GALON', 'Regular GTQ/GALON', 'Diesel GTQ/GALON']
precios = precios[precios_cols].copy()

### Verificacion y manejo de Nulos

Anteriormente se encontro que Ceras y tipo cambio eran las variables con nulos. Dado que ahora se seleccionaron las variables de interes antes, es probable que ya no existan nulos.

In [4]:
# Comprobar si hay valores nulos
print("Valores nulos en el dataset de consumo de combustibles:", consumos.isnull().sum().sum())
print("Valores nulos en el dataset de importación de combustibles:", importaciones.isnull().sum().sum())
print("Valores nulos en el dataset de precios de combustibles:", precios.isnull().sum().sum())

Valores nulos en el dataset de consumo de combustibles: 0
Valores nulos en el dataset de importación de combustibles: 0
Valores nulos en el dataset de precios de combustibles: 0


### Manejo de Fechas

Anteriormente se encontro que los 3 datasets contienen una unica variable culitativa la cual es `Fecha`. Esta se encuentra como string en español por lo que se debe de transformar a datetime con formato inglés para que pandas sea capaz de reconocer la variable como un objeto **datetime**

In [5]:
# Diccionario de meses en español a inglés
month_mapping = {
    'ene': 'Jan',
    'feb': 'Feb',
    'mar': 'Mar',
    'abr': 'Apr',
    'may': 'May',
    'jun': 'Jun',
    'jul': 'Jul',
    'ago': 'Aug',
    'sep': 'Sep',
    'oct': 'Oct',
    'nov': 'Nov',
    'dic': 'Dec'
}

# Función para reemplazar los nombres de los meses
def replace_months(date_str):
    for esp, eng in month_mapping.items():
        date_str = date_str.replace(esp, eng)
    return date_str

# Por fines practicos, se renombra la columna  'FECHA' a 'Fecha' en el dataset de precios de combustibles
precios.rename(columns={'FECHA': 'Fecha'}, inplace=True)

# Aplicar la función a las columnas de fecha de los dataframes
consumos['Fecha'] = consumos['Fecha'].apply(replace_months)
importaciones['Fecha'] = importaciones['Fecha'].apply(replace_months)
precios['Fecha'] = precios['Fecha'].apply(replace_months)

# Convertir las columnas de fecha a datetime
consumos['Fecha'] = pd.to_datetime(consumos['Fecha'], format='%b/%Y')
importaciones['Fecha'] = pd.to_datetime(importaciones['Fecha'], format='%b/%Y')
precios['Fecha'] = pd.to_datetime(precios['Fecha'], format='%d/%b/%Y')

# Establecer la columna 'Fecha' como índice en los tres dataframes
consumos.set_index('Fecha', inplace=True)
importaciones.set_index('Fecha', inplace=True)
precios.set_index('Fecha', inplace=True)

# Converir los indices a datetime
consumos.index = pd.to_datetime(consumos.index)
importaciones.index = pd.to_datetime(importaciones.index)
precios.index = pd.to_datetime(precios.index)

# Ordenar los índices
consumos.sort_index(inplace=True)
importaciones.sort_index(inplace=True)
precios.sort_index(inplace=True)

# Inferir la frecuencia de los índices
pd.infer_freq(consumos.index)
pd.infer_freq(importaciones.index)
pd.infer_freq(precios.index);

### Manejo de variables Diesel

Es necesario combinar ambas variables de diesel en una unica variable donde las fechas de antes del 2018 se tomaran para el diesel alto en azufre, posterior a 2018 se tomaran las de diesel alto en azufre.

In [6]:
def combine_diesel(df, high_sulfur_col, low_sulfur_col, combined_col):
    """
    Función que combina las columnas de Diesel de alto y bajo azufre en una sola columna.
    Toma los valores de Diesel de alto azufre hasta 2017 y los de Diesel de bajo azufre a partir de 2018.

    Args:
        df (pandas.DataFrame): El dataframe de entrada.
        high_sulfur_col (str): El nombre de la columna de Diesel de alto azufre.
        low_sulfur_col (str): El nombre de la columna de Diesel de bajo azufre.
        combined_col (str): El nombre de la nueva columna combinada.
    
    Returns:
        data (pandas.DataFrame): El dataframe de entrada con la nueva columna combinada.
    """
    # Crear una copia del dataframe
    data = df.copy()

    # Crear una nueva columna con los valores de Diesel de alto azufre
    data.loc[data.index < '2018-01-01', combined_col] = data[high_sulfur_col]
    data.loc[data.index >= '2018-01-01', combined_col] = data[low_sulfur_col]

    # Eliminar las columnas originales
    data.drop(columns=[high_sulfur_col, low_sulfur_col], inplace=True)

    return data

In [7]:
# Combina las columnas de Diesel de alto y bajo azufre en una sola columna
consumos = combine_diesel(consumos, 'Diesel alto azufre', 'Diesel bajo azufre', 'Diesel')
importaciones = combine_diesel(importaciones, 'Diesel alto azufre', 'Diesel bajo azufre', 'Diesel')

### Renombrar Variables

Por fines practicos se renombraran las variables para que sean más fácil de entender a que hacen referencia.

In [8]:
precios.rename(columns={
    'Superior GTQ/GALON': 'Gasolina Superior',
    'Regular GTQ/GALON': 'Gasolina Regular',
    'Diesel GTQ/GALON': 'Diesel',
    'Glp Cilindro 25Lbs GTQ/LB': 'Gas Propano'
}, inplace=True)

# Renombrar columnas en 'consumo_combustibles'
consumos.rename(columns={
    'Gasolina regular': 'Gasolina Regular',
    'Gasolina superior': 'Gasolina Superior',
    'Gas licuado de petróleo': 'Gas Propano'
}, inplace=True)

# Renombrar columnas en 'importacion_combustibles'
importaciones.rename(columns={
    'Gasolina regular': 'Gasolina Regular',
    'Gasolina superior': 'Gasolina Superior',
    'Gas licuado de petróleo': 'Gas Propano'
}, inplace=True)

In [9]:
consumos.head()

,Gasolina Regular,Gasolina Superior,Diesel
Fecha,,,
2000-01-01,202645.20,308156.82,634667.06
2000-02-01,205530.96,307766.31,642380.66
2000-03-01,229499.56,331910.29,699807.25
2000-04-01,210680.40,315648.08,586803.98
2000-05-01,208164.34,319667.97,656948.20


In [10]:
precios.head()

,Gasolina Superior,Gasolina Regular,Diesel
Fecha,,,
2021-01-01,21.91,21.11,17.61
2021-01-02,21.91,21.11,17.61
2021-01-03,21.91,21.11,17.61
2021-01-04,21.91,21.11,17.61
2021-01-05,21.91,21.11,17.61


In [11]:
importaciones.head()

,Gasolina Regular,Gasolina Superior,Diesel
Fecha,,,
2001-01-01,177776.50,373963.96,566101.99
2001-02-01,123115.99,243091.07,489525.80
2001-03-01,161726.42,312084.38,575559.68
2001-04-01,127338.74,285054.89,437745.42
2001-05-01,168730.19,300913.67,552609.13


### Guardar Data Limpia

In [12]:
# Guardar los dataframes procesados en nuevos archivos CSV
consumos.to_csv('data/consumos_clean.csv', sep=';', encoding='utf-8')
importaciones.to_csv('data/importaciones_clean.csv', sep=';', encoding='utf-8')
precios.to_csv('data/precios_clean.csv', sep=';', encoding='utf-8')
print("Dataframes procesados guardados en la carpeta 'data'.")

Dataframes procesados guardados en la carpeta 'data'.
